In [16]:
import pandas as pd
import pickle
from sklearn import linear_model
import numpy as np
import sklearn

def Toreffizienz_und_Passen(spieldaten, anfang,ende):
    
    mannschaft1=[]
    mannschaft2=[]
    tore= []
    schuesse=[]
    toreffizienz=[]
    passen=[]

    anfang=anfang*9
    ende=ende*9 
    count=0
    with (open(spieldaten, "rb")) as openfile:
        while True:
            try:
                if count< ende:
                    spieltag= pickle.load(openfile)
                    #print(spieltag)
                    #print(spieltag[1], spieltag[2], spieltag[4][0:2], spieltag[5][0:2])
                    if spieltag[1]=="Mannschaft1":
                        continue
                    #Mannschaften
                    mannschaft1.append(spieltag[1])
                    mannschaft2.append(spieltag[2])
                    #Effizienz
                    eff_heim= int(spieltag[4][0])/int(spieltag[4][1])
                    eff_gast= int(spieltag[5][0])/int(spieltag[5][1])
                    effi_gesamt= eff_heim-eff_gast
                    toreffizienz.append(effi_gesamt)
                    #Passen
                    passen_heim= int(spieltag[4][6][:-1])
                    passen_gast= int(spieltag[5][6][:-1])
                    passen_gesamt= passen_heim/passen_gast
                    passen.append(passen_gesamt)
                    data= {"Mannschaft1":mannschaft1,"Mannschaft2":mannschaft2,"Effizienz":toreffizienz,"Passen":passen}
                    dfmod= pd.DataFrame(data)
                    #print(dfmod.values)
                    #print(effi_gesamt)
                    count=count+1
                else:
                    break
            except EOFError:
                break
    df_visitor = pd.get_dummies(dfmod['Mannschaft2'], dtype=np.int64)
    df_home = pd.get_dummies(dfmod['Mannschaft1'], dtype=np.int64)


    '''Fehlerkorrektur in der Tabelle (NaN Fehler)'''
    nan_home=[]
    for (columnName, columnData) in df_home.iteritems():
       team= columnName
       if np.sum(columnData.values)>1:
            teamwerte= [team,columnData.values]
            nan_home.append(teamwerte)

    nan_gast=[]
    for (columnName, columnData) in df_visitor.iteritems():

       team= columnName
       if np.sum(columnData.values)>1:
            columnData = columnData*-1
            teamwerte= [team,columnData.values]
            nan_gast.append(teamwerte)
            #nan_gast.append(columnData.values)        

    '''Transformation: 1:Heimmannschaft, -1: Gastmannschaft und hinzunahme von Differenz'''
    df_model = df_home.sub(df_visitor)
    df_model['effizienz'] = dfmod['Effizienz']
    df_model['passen'] = dfmod['Passen']
    #print(df_model)
    '''
    Wenn eine Mannschaft nur Heimspiele hat muessen die Werte umgeschrieben,
    sonst nan in der Spalte und man kann nicht weiter arbeiten
    '''
    count=0
    for (columnName, columnData) in df_model.iteritems():

        if np.isnan(np.sum(columnData.values)):
            for i in nan_home:
                if columnName==i[0]:
                    df_model[columnName]= i[1]
            for i in nan_gast:
                if columnName==i[0]:
                    df_model[columnName]=i[1]
    #print(df_model)
    '''Verwendung von Ridge Regression
        wird verwendet wenn es viele Features besitzt oder wenige Trainingsdaten besitzt
        Vermeidung von Un- oder Ueberanpassung'''

    lr = sklearn.linear_model.Ridge(alpha=0.001) 
    X = df_model.drop(['effizienz','passen'],axis=1)
    y = df_model['passen','effizienz']
    lr.fit(X, y)
    
    df_ratings = pd.DataFrame(data={'team': X.columns, 'rating': lr.coef_})
    odds= df_ratings.values
    #print(odds)
    count2= 0
    ergebnisse=[]
    mannschaft1=[]
    mannschaft2=[]
    with (open(spieldaten, "rb")) as openfile:    
        while True:
            try:
                if count2< ende+9:
                    spieltag= pickle.load(openfile)
                    if spieltag[1]=="Mannschaft1":
                        continue
                    mannschaft1.append(spieltag[1])
                    mannschaft2.append(spieltag[2])
                    ergebnis= (spieltag[4][0:1][0])+":"+(spieltag[5][0:1][0])
                    ergebnisse.append(ergebnis)
                    count=count+1
                else:
                    if count2 <512:
                        count2= count+1
                    else:
                        break

            except EOFError:
                break
    mannschaft1= mannschaft1[ende:]
    mannschaft2= mannschaft2[ende:]
    ergebnisse= ergebnisse[ende:]

    gesamt=[]
    #print(odds)
    for i in range(0,9):

        for j in odds:
            if mannschaft1[i]== str(j[0]): 
                odd1=j[1]
            if mannschaft2[i]== str(j[0]):
                odd2=j[1]
        odd= [odd1,odd2]
        game=[mannschaft1[i], mannschaft2[i],ergebnisse[i]]
        gesamt.append(game+odd)
    return gesamt
liste= Toreffizienz_und_Passen("Statistik2019.pkl", 25,33)
for i in liste:
    print(i)


KeyError: ('passen', 'effizienz')